David Flowers II \
Z1942130 \
Assignment 8 \
Koop FA2024

# 1. Unemployment Rates (25 pts

## a. Highest Unemployment Rate (2010+) (10 pts)

In [1]:
import pandas as pd
import altair as alt

dataset = pd.read_csv("https://faculty.cs.niu.edu/~dakoop/cs503-2024fa/a8/illinois-employment.csv.gz")

# Change county column to uppercase
dataset["COUNTY"] = dataset["COUNTY"].str.upper()

# Filter highest unemployment rate since 2010
dataset_2010 = dataset[dataset.YEAR >= 2010]
highest_unemployement = dataset_2010.iloc[dataset_2010["RATE"].idxmax()]

print(f"Highest unemployment rate was {highest_unemployement.COUNTY} at {highest_unemployement.RATE}% in {highest_unemployement.YEAR}")

Highest unemployment rate was WINNEBAGO COUNTY at 14.1% in 2010


## b. Lowest Average Unemployment Rate in the 2000s (15 pts)

In [2]:
# Filter data down to just between 2000 - 2009
mask_2000_decade = (dataset.YEAR >= 2000) & (dataset.YEAR <= 2009)
dataset_2000s = dataset[mask_2000_decade]

# Group the county by its rate, and find the mean
averages = dataset_2000s.groupby("COUNTY")["RATE"].mean()

# Grab the min index (county) and min
min_avg_county = averages.idxmin()
min_avg_rate = averages.min()

print(f"The lowest average unemployement was {min_avg_county} with an average of {min_avg_rate}%")

The lowest average unemployement was BROWN COUNTY with an average of 3.62%


# 2. Employment Status over Time

## a. Line Chart (15 pts)

In [3]:
county_set = ["DEKALB COUNTY", 
              "KANE COUNTY COUNTY",
              "BOONE COUNTY",
              "MCHENRY COUNTY",
              "WINNEBAGO COUNTY",
              "OGLE COUNTY",
              "LEE COUNTY",
              "KENDALL COUNTY"]

filtered_dataset = dataset[dataset["COUNTY"].isin(county_set)]

# Pandas chart if you want that instead
# chart = filtered_dataset.pivot_table(index="YEAR", columns="COUNTY", values="RATE").plot(figsize=(10,6))

# Altair chart
chart = alt.Chart(filtered_dataset).mark_line().encode(
    x=alt.X("YEAR:Q", title="YEAR", axis=alt.Axis(format='d')),
    y=alt.Y("RATE:Q", title="RATE"),
    color=alt.Color("COUNTY:N", legend=alt.Legend(orient='right', title="County"))
    ).properties(width=640,height=480).configure_axis(grid=False).show()

alt.Chart(...)

## b. Stacked Bar Chart (15 pts)

In [4]:
# Copy original dataset, just in case
temp_dataset = dataset.copy()

# New Decade column to get averages
temp_dataset["DECADE"] = (dataset["YEAR"] // 10) * 10

decade_averages = temp_dataset.groupby("DECADE").agg({
    "LABOR_FORCE": "mean",
    "EMPLOYED": "mean"
}).reset_index()

# Create bar chart, specifying X, Y and dimensions
chart = alt.Chart(decade_averages).encode(
    x=alt.X("DECADE:O", title="Decade"),
    y=alt.Y("LABOR_FORCE:Q", title="Labor Force")
    ).properties(width=640, height=480)

# Color the bar chart
bar_chart = alt.layer(
    chart.mark_bar(color="blue"),
    chart.encode(y="EMPLOYED:Q").mark_bar(color="orange")).show()

alt.LayerChart(...)

# 3. Maps

## a. County Locations (20 pts)

In [5]:
data_url = "https://raw.githubusercontent.com/subyfly/topojson/refs/heads/master/us-states/IL-17-illinois-counties.json"
illinois_topo = alt.topo_feature(data_url, "cb_2015_illinois_county_20m")

# Copy the dataset, and make changes to it
tmp_dataset = dataset.copy()
tmp_dataset["fips_str"] = tmp_dataset["FIPS"].astype(str).str.zfill(3)
recent_data = tmp_dataset[(tmp_dataset.YEAR >= 2018) & (tmp_dataset.YEAR <= 2023)]

# Conditional for brushing
county_hover = alt.selection_point(on='mouseover', fields=["fips_str"], empty='none')

# Code below was adapted from the "altair example" from Vega-Altair
# https://altair-viz.github.io/gallery/us_incomebrackets_by_state_facet.html
alt.Chart(recent_data).mark_geoshape().encode(
    shape="geo:G",
    color="RATE:Q",
    tooltip=["COUNTY:N", "RATE:Q"],
    # Change stroke and stroke width on mouseover
    stroke=alt.when(county_hover).then(alt.value("red")).otherwise(alt.value("grey")),
    strokeWidth=alt.when(county_hover).then(alt.value(3)).otherwise(alt.value(1)),
    facet=alt.Facet("YEAR:Q", columns=3),
).transform_lookup(
    lookup="fips_str",
    from_=alt.LookupData(data=illinois_topo, key="properties.COUNTYFP"),
    as_="geo",
).project(
    type="mercator"
).properties(
    width=250
).add_params(
    county_hover
)

alt.Chart(...)